In [54]:
import datetime as dt
from datetime import date
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import time


def ticker_df(ticker, start, end):
    source = 'yahoo'
    df = web.DataReader(ticker, source, start, end)
    return df.drop(['High', 'Low', 'Open', 'Adj Close', 'Volume'], axis = 1)

def stock_correlation(ticker1, ticker2, start, end):
    stock1 = ticker_df(ticker1, start, end)
    stock2 = ticker_df(ticker2, start, end)
    correlation = stock1['Close'].pct_change().corr(stock2['Close'].pct_change())
    return(correlation)


In [68]:
def stock_correlation(stock1, stock2):
    correlation = stock1['Close'].pct_change().corr(stock2['Close'].pct_change())
    return(correlation)

In [84]:
start = dt.datetime(2018,1,1)
end = dt.datetime(2020,7,7)
ETFs = ['SPY', 'IWM', 'QQQ', 'GDX', 'GDXJ', 'GLD', 'SLV', 'USO', 'XLK', 'XLE', 'XRT', 'XRT', 'XLU', 'XLF']
stocks = ['MU', 'BA', 'ENPH', 'AMD', 'TSLA', 'BAC', 'JPM']
assets ={}
assets2 = {}

for ETF in ETFs:
    assets[ETF] = ticker_df(ETF, start, end)

for ETF in ETFs:
    assets2[ETF] = list(assets[ETF]['Close'])

df = pd.DataFrame(data = assets2, columns = assets.keys())
corr_matrix = df.corr()

for ETF1 in ETFs:
    for ETF2 in ETFs:
        corr_matrix[ETF1][ETF2] = stock_correlation(assets[ETF1], assets[ETF2])

In [85]:
corr_matrix

,SPY,IWM,QQQ,GDX,GDXJ,GLD,SLV,USO,XLK,XLE,XRT,XLU,XLF
SPY,1.000000,0.917689,0.947217,0.101234,0.200278,0.027448,0.241528,0.399859,0.955246,0.804302,0.833209,0.696182,0.917887
IWM,0.917689,1.000000,0.839013,0.117185,0.206882,0.031732,0.275883,0.377697,0.843867,0.828902,0.892533,0.610986,0.898629
QQQ,0.947217,0.839013,1.000000,0.075948,0.171765,0.034475,0.230178,0.352865,0.982011,0.680796,0.758116,0.574074,0.792264
GDX,0.101234,0.117185,0.075948,1.000000,0.951964,0.689546,0.549139,0.105775,0.057174,0.142957,0.094093,0.181447,0.059256
GDXJ,0.200278,0.206882,0.171765,0.951964,1.000000,0.690510,0.557360,0.112589,0.153311,0.203598,0.187673,0.276782,0.154042
GLD,0.027448,0.031732,0.034475,0.689546,0.690510,1.000000,0.762517,-0.020073,0.021739,0.002652,-0.013912,0.179682,-0.032740
SLV,0.241528,0.275883,0.230178,0.549139,0.557360,0.762517,1.000000,0.208020,0.222577,0.257838,0.213383,0.231704,0.193574
USO,0.399859,0.377697,0.352865,0.105775,0.112589,-0.020073,0.208020,1.000000,0.360969,0.576703,0.314931,0.146230,0.377860
XLK,0.955246,0.843867,0.982011,0.057174,0.153311,0.021739,0.222577,0.360969,1.000000,0.704558,0.758415,0.594691,0.819797
XLE,0.804302,0.828902,0.680796,0.142957,0.203598,0.002652,0.257838,0.576703,0.704558,1.000000,0.727531,0.514771,0.824572


In [ ]:
corr_matrix['']